# Pets classification

Tutorial : https://docs.fast.ai/tutorial.vision.html

In [ ]:
from fastai.data.transforms import get_image_files, RegexLabeller
from fastai.data.external import untar_data, URLs
from fastai.interpret import Interpretation
from jedi.inference.parser_cache import get_yield_exprs

path = untar_data(URLs.PETS)
files = get_image_files(path/"images")
len(files), path

In [ ]:
from fastcore.basics import using_attr
from fastai.data.block import DataBlock, CategoryBlock
from fastai.data.transforms import RandomSplitter
from fastai.vision.augment import Resize, aug_transforms
from fastai.vision.data import ImageDataLoaders, ImageBlock

pat = r'^(.*)_\d+.jpg'

dblock = DataBlock(
    blocks=[ImageBlock, CategoryBlock],
    get_items=get_image_files,
    splitter=RandomSplitter(seed=42),
    get_y=using_attr(RegexLabeller(pat), 'name'),
    item_tfms=[Resize(460)],
    batch_tfms=aug_transforms(size=224)
)
dblock.summary(path/'images')

In [ ]:
dls = dblock.dataloaders(path/'images')
dls.show_batch(nrows=1,ncols=3)

In [ ]:
from fastai.learner import Learner
from fastai.vision import models
from fastai.vision.learner import vision_learner
from fastai.metrics import error_rate
# noinspection PyUnresolvedReferences
from fastai.callback.schedule import lr_find

learn: Learner = vision_learner(dls, models.resnet34, metrics=error_rate)
lr = learn.lr_find()
lr

In [ ]:
learn.fine_tune(4, lr.valley)
learn.show_results()

In [ ]:
interp = Interpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(15,10))

In [ ]:
from fastai.interpret import ClassificationInterpretation

interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)